# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df.hvplot.points("Lng","Lat",geo=True, tiles = "EsriImagery", frame_width = 700, frame_height = 550, size = "Humidity", scale = 0.8, color = "City")

# Display the map
humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[(city_data_df["Max Temp"] >= 18) & (city_data_df["Max Temp"] <= 28) & 
                           (city_data_df["Wind Speed"] < 10)&
                           (city_data_df["Humidity"]>25)& (city_data_df["Humidity"]<55)]
# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
34,34,naberera,-4.2000,36.9333,21.12,54,94,6.38,TZ,1666108252
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df.loc[:, ["City", "Country", "Lat", "Lng"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,faya,SA,18.3851,42.4509,
34,naberera,TZ,-4.2000,36.9333,
63,banda,IN,25.4833,80.3333,
73,nalut,LY,30.3333,10.8500,
86,rafaela,AR,-31.2503,-61.4867,
102,rio gallegos,AR,-51.6226,-69.2181,
106,palencia,ES,42.4167,-4.5000,
114,yidu,CN,24.1614,115.1328,
170,sur,OM,22.5667,59.5289,
229,san cristobal,VE,7.7669,-72.2250,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 3000
limit = 20
params = {
    "radius" : radius,
    "apiKey":geoapify_key,
    "limit":limit,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
naberera - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
rafaela - nearest hotel: No hotel found
rio gallegos - nearest hotel: No hotel found
palencia - nearest hotel: No hotel found
yidu - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
san cristobal - nearest hotel: No hotel found
prieska - nearest hotel: No hotel found
san juan - nearest hotel: No hotel found
jalu - nearest hotel: No hotel found
kingman - nearest hotel: No hotel found
tandil - nearest hotel: No hotel found
viseu - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
barroso - nearest hotel: No hotel found
vryburg - nearest hotel: No hotel found
chivilcoy - nearest hotel: No hotel found
pali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
0,faya,SA,18.3851,42.4509,No hotel found
34,naberera,TZ,-4.2000,36.9333,No hotel found
63,banda,IN,25.4833,80.3333,No hotel found
73,nalut,LY,30.3333,10.8500,No hotel found
86,rafaela,AR,-31.2503,-61.4867,No hotel found
102,rio gallegos,AR,-51.6226,-69.2181,No hotel found
106,palencia,ES,42.4167,-4.5000,No hotel found
114,yidu,CN,24.1614,115.1328,No hotel found
170,sur,OM,22.5667,59.5289,No hotel found
229,san cristobal,VE,7.7669,-72.2250,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = narrowed_city_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    c="Humidity",
    cmap="viridis",
    colorbar=True,
    title="City Humidity",
    height=500,
    width=800,
    size=city_data_df["Humidity"]*10,
    alpha=0.5,
    hover_cols=["City", "Country", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,_size,City,Country,Country)